# Geonames
- Create function "fetch_geonames()".
- Download the geonames file provided in the API structure document.
- Turn the json file into a Pandas DataFrame "geonames" for future use.

In [ ]:
# Import the required libraries
import requests
import pandas as pd

# How do I change the function so that I do not need to add any arguments, so predefine the url and filename?

def fetch_geonames():
         
    # Define the URL for the JSON file
    url_geonames = "https://climateknowledgeportal.worldbank.org/themes/cckp/data/geonames.json"
    # Fetch the JSON data using a GET request
    response_geonames = requests.get(url_geonames)

    # Check if the request was successful
    if response_geonames.status_code == 200:
        # Parse the JSON response into a Python dictionary
        geonames_json = response_geonames.json()
        print("JSON-file fetched and parsed successfully!")
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        geonames_json = None

    # Flatten the nested structure into a list of dictionaries
    flat_geonames_data_json = [
        {
            "Country Code": country_code,
            "Country Name": details["N"],
            "Subnational Code": subnational_code,
            "Subnational Name": subnational_name
        }
        for country_code, details in geonames_json["country"].items()
        for subnational_code, subnational_name in details["S"].items()
    ]

    # Convert the list into a pandas DataFrame
    geonames_df = pd.DataFrame(flat_geonames_data_json)
    print("Geonames data flattened succesfully to a DataFrame!")

    # Return the DataFrame for further use
    print("Geonames dataframe returned for further use!")
    return geonames_df

    # Print "Geonames dataframe returned for further use!" once the function has run succesfully

    # Why does it not print?
    # The print statement is not executed because the function is returning the DataFrame before the print statement is reached.
    # Ah, so the "return" statement stops the function from running further?
    # Yes, the return statement exits the function and returns the specified value to the caller.


# Execute the function and fetch the geonames data
geonames_df = fetch_geonames()